# Conversational Task Agent

Taskbot that assist users in multi-step tasks, such as baking a 
birthday  cake  or  fixing  a  scratch  on  a  car  —  and  adapt  those  instructions  based  on  the 
resources  and  tools  available  to  the  customer.  If,  for  example,  a  customer  ran  out  of  an 
ingredient  halfway  through  a  recipe  or  didn’t  have  a  specific  tool  for  a  DIY  project,  the 
taskbot had to adjust the plan and suggest possible solutions.

## Phase 1 - (Task Retriever): Implementing a search index for recipes with OpenSearch

## Indexing

The Indexing.py module's purpose is to handle the index operations, might these be to calculate the embeddings and interact with the pickle files used to store the embeddings, to create the index and mappings or to parse and store the recipes from the base file to OpenSearch.

In [7]:
# Question Image
from modules.QuestionImage import QuestionImage
import requests
from PIL import Image
import pprint as pp

questionImg = QuestionImage()
img_url = "https://media.istockphoto.com/id/1319467946/photo/young-black-and-white-cow-heifer-in-a-meadow-looking-in-the-camera.jpg?s=612x612&w=0&k=20&c=Z1maGtrEMrbAEVw6ZTJwyvq2_rkolky9LJX34mSZ6Kg="

image = Image.open(requests.get(img_url, stream=True).raw)
question = "What animal is in the picture?"
answer = questionImg.ask(question, image)
display(image)
print(question)
print(answer)

KeyboardInterrupt: 

In [ ]:
from modules.SlotFilling import SlotFilling
import pprint as pp

slot = SlotFilling()

answer = slot.extract_slot("Yes. No. I''d like a salad with tomatos, lettuce and strawberries.", "What are the ingredients?")

pp.pprint(answer)

'tomatos, lettuce and strawberries'


In [4]:
from modules.IntentDetector import IntentDetector
import pprint as pp

intent = IntentDetector()
answer = intent.detect_intent("Hello")
pp.pprint(answer)

'YesIntent'


In [2]:
## Run Dialog Manaager
from DialogManager_TWIZ.dialog_factory.dialog_manager import DialogManager
from DialogManager_TWIZ.dialog_factory.dialog_elements import LaunchEvent
from DialogManager_TWIZ.states.start_state import StartState
from DialogManager_TWIZ.states.choose_type_state import ChooseTypeState
from DialogManager_TWIZ.states.display_recipe_state import DisplayRecipeState
from DialogManager_TWIZ.states.task_state import TaskState
from DialogManager_TWIZ.events.greetings_event import GreetingsEvent
from DialogManager_TWIZ.events.start_task_event import StartTaskEvent
from DialogManager_TWIZ.events.out_of_scope_event import OutOfScopeEvent


# create dialog manager
dialog_manager = DialogManager()

state = {
    "intent": "",
    "recipe": 0,
    "step": 0
}

def run_dialog_manager():
    response = dialog_manager.trigger(LaunchEvent(), state)
    print(response)

    while True:
        # receive intent from user utterance
        # its here that we should use IntentDetector
        state["intent"] = input("Intent: ")
        print(state["intent"])

        event = dialog_manager.event_type(state)
        print(event)
        response = dialog_manager.trigger(event, state)
        print(response)

run_dialog_manager() # run !


## STATES #########################################
LaunchState
StartState
ChooseTypeState
DisplayRecipeState
TaskState
###################################################
## EVENTS #########################################
LaunchEvent
ChooseType
NextEvent
GreetingsEvent
StartTaskEvent
OutOfScopeEvent
StopEvent
###################################################


## TRANSITIONS ### S-E->S #########################
LaunchState - LaunchEvent  ->  StartState
StartState - OutOfScope  ->  StartState
DisplayRecipeState - Stop  ->  StartState
TaskState - Stop  ->  StartState
StartState - Greetings  ->  ChooseTypeState
ChooseTypeState - OutOfScope  ->  ChooseTypeState
ChooseTypeState - ChooseType  ->  DisplayRecipeState
DisplayRecipeState - OutOfScope  ->  DisplayRecipeState
DisplayRecipeState - Next  ->  DisplayRecipeState
TaskState - OutOfScope  ->  TaskState
DisplayRecipeState - StartTask  ->  TaskState
###################################################

while: <DialogManager_TWIZ.dialog_

Exception: No event defined for the detected intent

In [1]:
## Run Dialog Manaager
from DialogManager_TWIZ.dialog_factory.dialog_manager import DialogManager
from DialogManager_TWIZ.dialog_factory.dialog_elements import LaunchEvent
from DialogManager_TWIZ.events.choose_type_event import ChooseType
from DialogManager_TWIZ.events.next_event import NextEvent
from DialogManager_TWIZ.events.stop_event import StopEvent
from DialogManager_TWIZ.states.start_state import StartState
from DialogManager_TWIZ.states.choose_type_state import ChooseTypeState
from DialogManager_TWIZ.states.display_recipe_state import DisplayRecipeState
from DialogManager_TWIZ.states.conversation_state import ConversationState
from DialogManager_TWIZ.states.end_state import EndState
from DialogManager_TWIZ.events.greetings_event import GreetingsEvent
from DialogManager_TWIZ.events.start_task_event import StartTaskEvent
from DialogManager_TWIZ.events.out_of_scope_event import OutOfScopeEvent
from modules.IntentDetector import IntentDetector
from modules.Search import Search
from modules.SlotFilling import SlotFilling
from modules.PlanLLMUtils import PlanLLMUtils
import json
import pprint as pp


dialog_manager = DialogManager()
intentDetector = IntentDetector()
search = Search()
slotFilling = SlotFilling()
pllm = PlanLLMUtils()

state = {
    "intent": "",
    "userInput": "",
    "recipeCount": 0,
    "step": 0,
    "slotFilling" : slotFilling,
    "search" : search,
    "recipes" : [],
    "conversationJSON" : {"dialog": []},
    "pllm" : pllm
}

def map_intents_to_events(intent):
    intent_event_mapping = {
        # ChooseTypeEvent
        "SelectIntent": ChooseType,
        "IdentifyRestrictionsIntent": ChooseType,
        "AdjustServingsIntent": ChooseType,
        "NoRestrictionsIntent": ChooseType,
        "SubstitutionIntent": ChooseType,
        "IdentifyProcessIntent": ChooseType,
        "HelpIntent": ChooseType,
        
        # GreetingsEvent
        "GreetingIntent": GreetingsEvent,
        "ChitChatIntent": GreetingsEvent,
        
        # NextEvent
        "MoreOptionsIntent": NextEvent,
        "RepeatIntent": NextEvent,
        "GoToStepIntent": NextEvent,
        "PreviousStepIntent": NextEvent,
        "SuggestionsIntent": NextEvent,
        "IngredientsConfirmationIntent": NextEvent,
        "NextStepIntent": NextEvent,
        "YesIntent": NextEvent,
        
        # OutOfScopeEvent
        "GetCuriositiesIntent": OutOfScopeEvent,
        "ProvideUserNameIntent": OutOfScopeEvent,
        "QuestionIntent": OutOfScopeEvent,
        "MoreDetailIntent": OutOfScopeEvent,
        "OutOfScopeIntent": OutOfScopeEvent,
        "FallbackIntent": OutOfScopeEvent,
        "NoneOfTheseIntent": OutOfScopeEvent,
        "InappropriateIntent": OutOfScopeEvent,
        
        # StartTaskEvent
        "ShowStepsIntent": StartTaskEvent,
        "SetTimerIntent": StartTaskEvent,
        "ShoppingIntent": StartTaskEvent,
        "StartStepsIntent": StartTaskEvent,
        "ResumeTaskIntent": StartTaskEvent,
        
        # StopEvent
        "TerminateCurrentTaskIntent": StopEvent,
        "CompleteTaskIntent": StopEvent,
        "PauseIntent": StopEvent,
        "CancelIntent": StopEvent,
        "NoIntent": StopEvent,
        "StopIntent": StopEvent,
    }
    return intent_event_mapping[intent]

def run_dialog_manager():
    response = dialog_manager.trigger(LaunchEvent(), state)
    print(response['response'])

    while True:
        # receive intent from user utterance
        # its here that we should use IntentDetector
        userInput = input("Intent: ")
        state["userInput"] = userInput
        
        intent = intentDetector.detect_intent(userInput)
        state["intent"] = map_intents_to_events(intent).id
        # print(state["intent"])

        event = dialog_manager.event_type(state)
        print("Event: ",event)

        currState = dialog_manager.get_state()
        print("State: ", currState)

        if (currState == "EndState"):
            confirm = input("(yes/no): ")
            if confirm == "yes":
                break
            else:
                state["intent"] = "SelectIntent"
            
        # check this lmao
        if ((event.id != "Stop" and currState == "ConversationState") or (event.id == "StartTask" and currState == "DisplayRecipeState")): 
            state["conversationJSON"] = pllm.add_to_json(state["conversationJSON"], "user", userInput, state["step"])

        response = dialog_manager.trigger(event, state)

        if ((event.id != "Stop" and currState == "ConversationState")  or (event.id == "StartTask" and currState == "DisplayRecipeState")): 
            print(json.loads(response['response'].replace("'", "\""))['dialog'][state["step"]]["system"])
        else:
            print(response['response'])

        if ((event.id != "Stop" and currState == "ConversationState") or (event.id == "StartTask" and currState == "DisplayRecipeState")):
            state["conversationJSON"] = json.loads(response['response'].replace("'", "\""))
            state["step"] += 1

run_dialog_manager() # run !


Hello, how can I help you search for a recipe?
Event:  <class 'DialogManager_TWIZ.events.stop_event.StopEvent'>
State:  StartState
What type of recipe do you want?
Event:  <class 'DialogManager_TWIZ.events.choose_type_event.ChooseType'>
State:  ChooseTypeState
Great option!
Here is the top suggested recipe:Lasagna Roses. Feel free to ask for the next recipe.
Event:  <class 'DialogManager_TWIZ.events.start_task_event.StartTaskEvent'>
State:  DisplayRecipeState
Ready, set, go! Step 1: Preheat oven to 375\u02daF (190\u02daC). 

Event:  <class 'DialogManager_TWIZ.events.next_event.NextEvent'>
State:  ConversationState
Step 2: Heat olive oil in a large pot over medium heat. Add onions and garlic. Cook until softened. 

Event:  <class 'DialogManager_TWIZ.events.out_of_scope_event.OutOfScopeEvent'>
State:  ConversationState
Are you still with me? Step 3: Add the ground beef, breaking it apart with a spoon, and stir until browned. 

Event:  <class 'DialogManager_TWIZ.events.stop_event.StopEven

In [ ]:
from modules.Indexing import Indexing


indexer = Indexing()

indexer.calculateAndStoreEmbeddings()
print("Embeddings stored and encoded")
indexer.createOpenSearchMappings()
print("Mappings created")
indexer.readAndStoreRecipesFromFile("./receitas.json")
print("Recipes stored")

## Search

The Search.py module is simply an abstraction of the SearchBuilder.py module, made to simplify the search operations for a user. It's easily scalable thanks to the SearchBuilder layer and was made to be used as a simple interface for the system.

In [2]:
from modules.Search import Search

search = Search()

# search.SearchTitleAndDescriptionTxt("chicken curry")
# search.SearchRecipeTime(30, 5)
# search.SearchRecipeNameTime(30, 5, "chicken")
# search.SearchRecipeIngredients("chicken curry")
# search.SearchRecipeExcludeIngredients("curry", "chicken")
# search.SearchRecipeDifficulty("easy")
# search.SearchRecipeNameDifficulty("easy", "chicken")
search.SearchTitleEmbeddings("chicken")
# search.SearchDescriptionEmbeddings("chicken")
# search.SearchSingleTitleAndDescriptionTxtInstructions("chicken curry")

{'took': 39,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 44, 'relation': 'eq'},
  'max_score': 1.6324937,
  'hits': [{'_index': 'user212',
    '_type': '_doc',
    '_id': '557',
    '_score': 1.6324937,
    '_source': {'doc_id': 557}},
   {'_index': 'user212',
    '_type': '_doc',
    '_id': '556',
    '_score': 1.6324937,
    '_source': {'doc_id': 556}},
   {'_index': 'user212',
    '_type': '_doc',
    '_id': '559',
    '_score': 1.6324937,
    '_source': {'doc_id': 559}},
   {'_index': 'user212',
    '_type': '_doc',
    '_id': '561',
    '_score': 1.6324937,
    '_source': {'doc_id': 561}},
   {'_index': 'user212',
    '_type': '_doc',
    '_id': '35',
    '_score': 1.617717,
    '_source': {'doc_id': 35}}]}}

## SearchBuilder

The Search Builder API is meant to be used as a tool to easily and in an iterative manner build simple or complex search OpenSearch queries. This module avoids repetitive code, also as a way to ensure good application scaling and coding practices and maintaining a developer-friendly interface.

In [ ]:
import pprint as pp
from modules.SearchBuilder import SearchBuilder
from PIL import Image
import requests

searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(["Japanese"])
searchBuilder.setOptionalTags(["Sukiaki"])
searchBuilder.setIngredients("soy")
searchBuilder.excludeIngredients("chicken")
searchBuilder.setTimeFrame(150, 5)
result = searchBuilder.Search("beef Sukiyaki")
pp.pprint(result)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '749',
                    '_index': 'user212',
                    '_score': 14.587381,
                    '_source': {'contents': "{'displayName': 'Beef Trim "
                                            "Sukiyaki', 'description': None, "
                                            "'canonicalName': 'Beef Trim "
                                            "Sukiyaki', 'prepTimeMinutes': "
                                            "None, 'cookTimeMinutes': None, "
                                            "'totalTimeMinutes': 150, "
                                            "'cookingMethod': None, "
                                            "'difficultyLevel': None, "
                                            "'images': [{'url': "
                                            "'https://m.media-amazon.com/images/S/alexa-kitchen-msa-na-prod/recipes/seriouseats/8e08703e3ebeefc4d0a4ad8

In [ ]:
# search for vegetarian pesto pasta
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(['Vegan', 'Vegetarian'])
searchBuilder.setOptionalTags(['Gluten Free'])

response = searchBuilder.Search("Pesto pasta")
pp.pprint(response) # print

In [ ]:
# search for Yaki udon
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("mushrooms")
searchBuilder.excludeIngredients("brocolli")
searchBuilder.setOptionalTags(["Vegetarian"])
searchBuilder.setTimeFrame(20, 15)

response = searchBuilder.Search("Yaki udon with at home ingredients")
pp.pprint(response)# print

In [ ]:
# search for Dumpling Miso Soup Ramen
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("dumplings")
searchBuilder.setMandatoryTags(["Vegan", "Vegetarian", "Gluten Free"])
searchBuilder.setOptionalTags(["Main Dishes", "Japanese"])

response = searchBuilder.Search("Dumpling Miso Soup Ramen")
pp.pprint(response)# print

In [ ]:
# search for Ceasar Salad
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setOptionalTags(["Main Dishes"])

response = searchBuilder.SearchByDescriptionEmbeddings("Ceasar Salad")
pp.pprint(response)# print

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '564',
                    '_index': 'user212',
                    '_score': 2.366397,
                    '_source': {'contents': "{'displayName': 'New York Strip "
                                            "Steak with Chimichurri', "
                                            "'description': 'Rich, tender "
                                            'strip steak drizzled with a '
                                            'classic herb chimichurri sauce '
                                            'makes a great dinner for two, but '
                                            'you can also double the '
                                            'ingredients to ensure leftovers '
                                            'for sandwiches later in the week. '
                                            'The sauce can be made a day '
                                            'ahe

In [ ]:
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()

lasagna_url = "https://www.unileverfoodsolutions.pt/dam/global-ufs/mcos/portugal/calcmenu/recipes/PT-recipes/In-Development/lasanha/main-header.jpg"

response = searchBuilder.SearchByImageEmbeddings([Image.open(requests.get(lasagna_url, stream=True).raw)])
pp.pprint(response)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '308',
                    '_index': 'user212',
                    '_score': 1.834137,
                    '_source': {'contents': "{'displayName': 'Lasagna "
                                            "Bolognese for Two', "
                                            "'description': None, "
                                            "'canonicalName': 'Lasagna "
                                            "Bolognese for Two', "
                                            "'prepTimeMinutes': None, "
                                            "'cookTimeMinutes': None, "
                                            "'totalTimeMinutes': 70, "
                                            "'cookingMethod': None, "
                                            "'difficultyLevel': 'easy', "
                                            "'images': [{'url': "
                                           

In [ ]:
from opensearchpy import OpenSearch

host = 'api.novasearch.org'
port = 443
user = 'user212'
password = 'soO-2518'
index_name = user
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

## Closing the client connection

In [ ]:
be sure you want to close the connection before running this code

resp = client.indices.close(index = index_name, timeout=600)
print(resp)

## Deleting the Index

In [ ]:
be sure you want to delete the index before running this code

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = 600
    )
    print('\nDeleting index:')
    print(response)


Deleting index:
{'acknowledged': True}
